# Lifecycle of a task

We start out detailing the full lifecylce of a task, from when it is created and submitted till when it is completed and the results are returned to the user. 

## 10,000 feet view

We have a python function convenitenly named `expensive_computation` which executes an expensive computation. To keep it simple all it does is sleep for a given number of seconds.

It gets called in sequence a number of times (`n_runs`) to be specific

In [1]:
n_runs = 30

def expensive_computation():
    import time
    time.sleep(1)
    return 1

results = [expensive_computation() for _ in range(n_runs)]
assert sum(results) == n_runs

Below is the execution visualized

<img src="sequential_simple_.jpeg" height=300>

We want to:
- Run the same function in a distributed fashion - i.e. in parallel on a cluster of machines
- Get the results of the function as they become available

We do this by following these steps:
- Convert the `expensive_computation` function to a ray task decoration by decorating it with `ray.remote`
- Submit a task for execution by calling `future = expensive_computation.remote()`
- Use the returned `future` object reference to fetch the result of the function by calling `ray.get(future)` 

In [2]:
import ray
import time


@ray.remote  # decorator to convert python function to ray task
def expensive_computation():
    time.sleep(1)
    return 1


# submit n_run ray tasks to a ray cluster
# and keep a reference to the task futures
futures = [expensive_computation.remote() for _ in range(n_runs)]

# wait for all tasks to complete and get the resulting objects
results = ray.get(futures)

# confirm that we got the right result
assert sum(results) == n_runs

2023-11-08 10:38:08,381	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Here is what is happening under the hood:

<img src="parallel_simple.jpeg" height="300">

## 1000 feet view

Let's detail the parallel execution of the function a bit more.

More specifically:
- ray tasks are executed on a ray cluster as part of a ray job
- ray workers are the processes that execute the tasks
- futures in ray are called `ObjectRef`s short for object references
- results are stored as objects in the ray object store
- ray.get() is used to fetch an object given its object reference

In [3]:
import ray
import time


@ray.remote
def expensive_computation():
    time.sleep(1)
    return 1


# this returns an object reference to the result
object_ref_future = expensive_computation.remote()

Here we inspect the object reference and see that it is a ray.ObjectRef at a given address

In [4]:
object_ref_future

ObjectRef(a631fe8d231813bfffffffffffffffffffffffff0100000001000000)

We then call `ray.get` which waits till the object reference is resolved and returns the resulting value

In [5]:
object_value = ray.get(object_ref_future) 
object_value

1

Here is a more detailed view of the parallel execution


<img src="parallel_1000_feet.jpeg" height="300">

Let's use the ray state client to verify the above.

We re-declare the `expensive_computation` but give it a unique name so we can easily track its state and a longer sleep time so we can see the state evolve more clearly

In [6]:
from uuid import uuid4

task_sleep_time = 20


@ray.remote
def my_task():
    import time

    time.sleep(task_sleep_time)
    return 1


id_ = str(uuid4())[:8]
name = f"expensive_computation_{id_}"
ray_task = my_task.options(name=name)

We submit the task

In [7]:
ray_task.remote()

ObjectRef(79cc316456d39201ffffffffffffffffffffffff0100000001000000)

We now request the cluster state to see our task running

In [8]:
from ray.util.state import list_tasks
import time

start_time = time.time()

while time.time() - start_time < task_sleep_time:
    time.sleep(5)
    task = next(task for task in list_tasks() if task.name == name)
    print(
        f"task {task.name} is in state={task.state} running on worker {task.worker_id[:8]} as part of Job ID {task.job_id}"
    )

task expensive_computation_550fb086 is in state=RUNNING running on worker 4f49061b as part of Job ID 01000000
task expensive_computation_550fb086 is in state=RUNNING running on worker 4f49061b as part of Job ID 01000000
task expensive_computation_550fb086 is in state=RUNNING running on worker 4f49061b as part of Job ID 01000000
task expensive_computation_550fb086 is in state=RUNNING running on worker 4f49061b as part of Job ID 01000000


## 100 feet view

Let's further detail the lifecycle of a ray task.

More specifically here is what a cluster looks like:


<img src="ray_cluster_.jpeg" height="500">

Things to keep in mind:

- The head node is a special node that runs the driver and the global control service. 
- The head node can also spawn worker processes to execute tasks
- The Global control service keeps track of cluster state that is not supposed to change often
- Each worker process will keep track of all the task it executes and submits in its ownership table
- Small objects (< 100KB) are stored in the in-process object store of a worker
- Large objects are stored in the plasma store which is shared across worker processes on the same node
- plasma store by default is in-memory and takes up 30% of the memory of the node
- if plasma store is full, objects are spilled to disk

With the cluster architecture in mind, let's look at the lifecycle of a task in more detail.

<img src="parallel_100feet.jpeg" height="400">

### Object management and dependency resolution

Let's drill down on how a task's dependencies are resolved - using the following example of simple batch inference:

- we load a model
- we use the model to make predictions on an input

In [9]:
import ray
import numpy as np

def load_model(size_mb):
    weights = np.ones((1024, 1024, size_mb), dtype=np.uint8)
    assert weights.nbytes / 1024**2 == size_mb
    return weights


@ray.remote
def predict(model, input):
    return model * input


We start with this simple implementation

In [10]:
# load 1 GB model in memory
model = load_model(1_000) 

# submit 3 tasks to the cluster
futures = ray.get([predict.remote(model, i) for i in range(3)])

There are 3 `predict` tasks that will be submitted.

- The owner of each task will need to go over all the task arguments and:
    - check that all the arguments are available
    - store a reference to all the available arguments in the (plasma) shared or inprocess object store
- In the case of our 1 GB model, the owner will make use of the shared object store given our object is > 1KB
- Each owner will create a copy of the model and produce an object reference to use as the argument for the task
- Each owner process will now execute their task

The outcome is that we have made 3 copies of the model in the shared object store.

Instead to save on memory, we should use the `ray.put` API to store the model in the shared object store and pass the reference to the model as an argument to the task.

Here is the optimized implementation:


In [11]:
# put the model in the object store and get a reference to it
model_ref = ray.put(model)

# submit 3 tasks to the cluster using the same model reference
futures = ray.get([predict.remote(model_ref, i) for i in range(3)])

## 10 feet view of ray

### Inspecting debug logs

Given the below code, we can inspect the debug logs to see what is happening under the hood

In [ ]:
import ray
import numpy as np


def load_model(size_mb):
    weights = np.ones((1024, 1024, size_mb), dtype=np.uint8)
    assert weights.nbytes / 1024**2 == size_mb
    return weights


@ray.remote
def predict(model, input):
    return model * input


model = load_model(size_mb=1000)
obj_ref = predict.remote(model, 1)
ray.get(obj_ref)  # c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000

In [29]:
import pandas as pd


def color_by_log_type(row):
    colors = {
        "Initialization": "background-color: grey",
        "Create object": "background-color: green",
        "Handle Return Object": "background-color: green",
        "Construct core worker process": "background-color: green",
        "Construct core worker": "background-color: green",
        "Polling": "background-color: grey",
        "Receive Task": "background-color: yellow",
        "Add Task": "background-color: yellow",
        "Execute Task": "background-color: orange",
        "Updating Reference Count": "background-color: grey",
        "Delete object": "background-color: red",
        "Finished Task": "background-color: green",
    }
    return [colors.get(row["log_type"], "")] * len(row)


df_debug_logs = pd.read_csv("debugging_ray.csv")

styler = df_debug_logs.style.apply(color_by_log_type, axis=1)
html = styler.to_html()

# Save the HTML to a file
with open("styled_debug_logs.html", "w") as f:
    f.write(html)

# Display the HTML on GitHub (must be placed within backticks)
print(f"`{html}`")

styler

`<style type="text/css">
#T_0f195_row0_col0, #T_0f195_row0_col1, #T_0f195_row0_col2, #T_0f195_row0_col3, #T_0f195_row0_col4, #T_0f195_row0_col5, #T_0f195_row0_col6, #T_0f195_row0_col7, #T_0f195_row6_col0, #T_0f195_row6_col1, #T_0f195_row6_col2, #T_0f195_row6_col3, #T_0f195_row6_col4, #T_0f195_row6_col5, #T_0f195_row6_col6, #T_0f195_row6_col7, #T_0f195_row14_col0, #T_0f195_row14_col1, #T_0f195_row14_col2, #T_0f195_row14_col3, #T_0f195_row14_col4, #T_0f195_row14_col5, #T_0f195_row14_col6, #T_0f195_row14_col7, #T_0f195_row25_col0, #T_0f195_row25_col1, #T_0f195_row25_col2, #T_0f195_row25_col3, #T_0f195_row25_col4, #T_0f195_row25_col5, #T_0f195_row25_col6, #T_0f195_row25_col7, #T_0f195_row32_col0, #T_0f195_row32_col1, #T_0f195_row32_col2, #T_0f195_row32_col3, #T_0f195_row32_col4, #T_0f195_row32_col5, #T_0f195_row32_col6, #T_0f195_row32_col7, #T_0f195_row33_col0, #T_0f195_row33_col1, #T_0f195_row33_col2, #T_0f195_row33_col3, #T_0f195_row33_col4, #T_0f195_row33_col5, #T_0f195_row33_col6, #T_0

,timestamp,log_level,process_id,thread_id,source_file,line_number,message,log_type
0,"2023-11-08 22:23:33,076",I,90551,868233,core_worker_process.cc,107,Constructing CoreWorkerProcess. pid: 90551,Construct core worker process
1,"2023-11-08 22:23:33,077",D,90551,868239,core_worker_process.cc,193,"Getting system config from raylet, remaining retries = 10",Initialization
2,"2023-11-08 22:23:33,077",D,90551,868233,ray_config.cc,70,"RayConfig is initialized with: is_external_storage_type_fs=true,object_spilling_config=""{\""type\"": \""filesystem\"", \""params\"": {\""directory_path\"": \""/tmp/ray/session_2023-11-08_22-23-28_636183_90530\""}}"",",Initialization
3,"2023-11-08 22:23:33,077",D,90551,868233,stats.h,78,Initialized stats,Initialization
4,"2023-11-08 22:23:33,077",I,90551,868233,io_service_pool.cc,35,IOServicePool is running with 1 io_service.,Initialization
5,"2023-11-08 22:23:33,077",D,90551,868233,metrics_agent_client.h,41,Initiate the metrics client of address:127.0.0.1 port:65188,Initialization
6,"2023-11-08 22:23:33,078",D,90551,868233,core_worker.cc,131,"Constructing CoreWorker, worker_id: 908b09e13a2ad715574e2cc981e4e890eeaae7383173260e5ed4e2e8",Construct core worker
7,"2023-11-08 22:23:33,078",I,90551,868233,grpc_server.cc,129,"worker server started, listening on port 10004.",Initialization
8,"2023-11-08 22:23:33,079",I,90551,868233,core_worker.cc,227,"Initializing worker at address: 127.0.0.1:10004, worker ID 908b09e13a2ad715574e2cc981e4e890eeaae7383173260e5ed4e2e8, raylet 3c3f9621d77d3aed914fa9d0cd9d85b1e58c16337101a2b005e70fd2",Initialization
9,"2023-11-08 22:23:33,080",D,90551,868233,gcs_client.cc,134,GcsClient connected 127.0.0.1:6379,Initialization


<style type="text/css">
#T_69520_row0_col0, #T_69520_row0_col1, #T_69520_row0_col2, #T_69520_row0_col3, #T_69520_row0_col4, #T_69520_row0_col5, #T_69520_row0_col6, #T_69520_row0_col7, #T_69520_row6_col0, #T_69520_row6_col1, #T_69520_row6_col2, #T_69520_row6_col3, #T_69520_row6_col4, #T_69520_row6_col5, #T_69520_row6_col6, #T_69520_row6_col7, #T_69520_row14_col0, #T_69520_row14_col1, #T_69520_row14_col2, #T_69520_row14_col3, #T_69520_row14_col4, #T_69520_row14_col5, #T_69520_row14_col6, #T_69520_row14_col7, #T_69520_row25_col0, #T_69520_row25_col1, #T_69520_row25_col2, #T_69520_row25_col3, #T_69520_row25_col4, #T_69520_row25_col5, #T_69520_row25_col6, #T_69520_row25_col7, #T_69520_row32_col0, #T_69520_row32_col1, #T_69520_row32_col2, #T_69520_row32_col3, #T_69520_row32_col4, #T_69520_row32_col5, #T_69520_row32_col6, #T_69520_row32_col7, #T_69520_row33_col0, #T_69520_row33_col1, #T_69520_row33_col2, #T_69520_row33_col3, #T_69520_row33_col4, #T_69520_row33_col5, #T_69520_row33_col6, #T_69520_row33_col7, #T_69520_row34_col0, #T_69520_row34_col1, #T_69520_row34_col2, #T_69520_row34_col3, #T_69520_row34_col4, #T_69520_row34_col5, #T_69520_row34_col6, #T_69520_row34_col7, #T_69520_row35_col0, #T_69520_row35_col1, #T_69520_row35_col2, #T_69520_row35_col3, #T_69520_row35_col4, #T_69520_row35_col5, #T_69520_row35_col6, #T_69520_row35_col7, #T_69520_row36_col0, #T_69520_row36_col1, #T_69520_row36_col2, #T_69520_row36_col3, #T_69520_row36_col4, #T_69520_row36_col5, #T_69520_row36_col6, #T_69520_row36_col7, #T_69520_row47_col0, #T_69520_row47_col1, #T_69520_row47_col2, #T_69520_row47_col3, #T_69520_row47_col4, #T_69520_row47_col5, #T_69520_row47_col6, #T_69520_row47_col7 {
  background-color: green;
}
#T_69520_row1_col0, #T_69520_row1_col1, #T_69520_row1_col2, #T_69520_row1_col3, #T_69520_row1_col4, #T_69520_row1_col5, #T_69520_row1_col6, #T_69520_row1_col7, #T_69520_row2_col0, #T_69520_row2_col1, #T_69520_row2_col2, #T_69520_row2_col3, #T_69520_row2_col4, #T_69520_row2_col5, #T_69520_row2_col6, #T_69520_row2_col7, #T_69520_row3_col0, #T_69520_row3_col1, #T_69520_row3_col2, #T_69520_row3_col3, #T_69520_row3_col4, #T_69520_row3_col5, #T_69520_row3_col6, #T_69520_row3_col7, #T_69520_row4_col0, #T_69520_row4_col1, #T_69520_row4_col2, #T_69520_row4_col3, #T_69520_row4_col4, #T_69520_row4_col5, #T_69520_row4_col6, #T_69520_row4_col7, #T_69520_row5_col0, #T_69520_row5_col1, #T_69520_row5_col2, #T_69520_row5_col3, #T_69520_row5_col4, #T_69520_row5_col5, #T_69520_row5_col6, #T_69520_row5_col7, #T_69520_row7_col0, #T_69520_row7_col1, #T_69520_row7_col2, #T_69520_row7_col3, #T_69520_row7_col4, #T_69520_row7_col5, #T_69520_row7_col6, #T_69520_row7_col7, #T_69520_row8_col0, #T_69520_row8_col1, #T_69520_row8_col2, #T_69520_row8_col3, #T_69520_row8_col4, #T_69520_row8_col5, #T_69520_row8_col6, #T_69520_row8_col7, #T_69520_row9_col0, #T_69520_row9_col1, #T_69520_row9_col2, #T_69520_row9_col3, #T_69520_row9_col4, #T_69520_row9_col5, #T_69520_row9_col6, #T_69520_row9_col7, #T_69520_row10_col0, #T_69520_row10_col1, #T_69520_row10_col2, #T_69520_row10_col3, #T_69520_row10_col4, #T_69520_row10_col5, #T_69520_row10_col6, #T_69520_row10_col7, #T_69520_row11_col0, #T_69520_row11_col1, #T_69520_row11_col2, #T_69520_row11_col3, #T_69520_row11_col4, #T_69520_row11_col5, #T_69520_row11_col6, #T_69520_row11_col7, #T_69520_row12_col0, #T_69520_row12_col1, #T_69520_row12_col2, #T_69520_row12_col3, #T_69520_row12_col4, #T_69520_row12_col5, #T_69520_row12_col6, #T_69520_row12_col7, #T_69520_row13_col0, #T_69520_row13_col1, #T_69520_row13_col2, #T_69520_row13_col3, #T_69520_row13_col4, #T_69520_row13_col5, #T_69520_row13_col6, #T_69520_row13_col7, #T_69520_row15_col0, #T_69520_row15_col1, #T_69520_row15_col2, #T_69520_row15_col3, #T_69520_row15_col4, #T_69520_row15_col5, #T_69520_row15_col6, #T_69520_row15_col7, #T_69520_row16_col0, #T_69520_row16_col1, #T_69520_row16_col2, #T_69520_row16_col3, #T_69520_row16_col4, #T_69520_row16_col5, #T_69520_row16_col6, #T_69520_row16_col7, #T_69520_row17_col0, #T_69520_row17_col1, #T_69520_row17_col2, #T_69520_row17_col3, #T_69520_row17_col4, #T_69520_row17_col5, #T_69520_row17_col6, #T_69520_row17_col7, #T_69520_row18_col0, #T_69520_row18_col1, #T_69520_row18_col2, #T_69520_row18_col3, #T_69520_row18_col4, #T_69520_row18_col5, #T_69520_row18_col6, #T_69520_row18_col7, #T_69520_row19_col0, #T_69520_row19_col1, #T_69520_row19_col2, #T_69520_row19_col3, #T_69520_row19_col4, #T_69520_row19_col5, #T_69520_row19_col6, #T_69520_row19_col7, #T_69520_row20_col0, #T_69520_row20_col1, #T_69520_row20_col2, #T_69520_row20_col3, #T_69520_row20_col4, #T_69520_row20_col5, #T_69520_row20_col6, #T_69520_row20_col7, #T_69520_row21_col0, #T_69520_row21_col1, #T_69520_row21_col2, #T_69520_row21_col3, #T_69520_row21_col4, #T_69520_row21_col5, #T_69520_row21_col6, #T_69520_row21_col7, #T_69520_row26_col0, #T_69520_row26_col1, #T_69520_row26_col2, #T_69520_row26_col3, #T_69520_row26_col4, #T_69520_row26_col5, #T_69520_row26_col6, #T_69520_row26_col7, #T_69520_row27_col0, #T_69520_row27_col1, #T_69520_row27_col2, #T_69520_row27_col3, #T_69520_row27_col4, #T_69520_row27_col5, #T_69520_row27_col6, #T_69520_row27_col7, #T_69520_row28_col0, #T_69520_row28_col1, #T_69520_row28_col2, #T_69520_row28_col3, #T_69520_row28_col4, #T_69520_row28_col5, #T_69520_row28_col6, #T_69520_row28_col7, #T_69520_row29_col0, #T_69520_row29_col1, #T_69520_row29_col2, #T_69520_row29_col3, #T_69520_row29_col4, #T_69520_row29_col5, #T_69520_row29_col6, #T_69520_row29_col7, #T_69520_row30_col0, #T_69520_row30_col1, #T_69520_row30_col2, #T_69520_row30_col3, #T_69520_row30_col4, #T_69520_row30_col5, #T_69520_row30_col6, #T_69520_row30_col7, #T_69520_row31_col0, #T_69520_row31_col1, #T_69520_row31_col2, #T_69520_row31_col3, #T_69520_row31_col4, #T_69520_row31_col5, #T_69520_row31_col6, #T_69520_row31_col7, #T_69520_row37_col0, #T_69520_row37_col1, #T_69520_row37_col2, #T_69520_row37_col3, #T_69520_row37_col4, #T_69520_row37_col5, #T_69520_row37_col6, #T_69520_row37_col7, #T_69520_row38_col0, #T_69520_row38_col1, #T_69520_row38_col2, #T_69520_row38_col3, #T_69520_row38_col4, #T_69520_row38_col5, #T_69520_row38_col6, #T_69520_row38_col7, #T_69520_row39_col0, #T_69520_row39_col1, #T_69520_row39_col2, #T_69520_row39_col3, #T_69520_row39_col4, #T_69520_row39_col5, #T_69520_row39_col6, #T_69520_row39_col7, #T_69520_row40_col0, #T_69520_row40_col1, #T_69520_row40_col2, #T_69520_row40_col3, #T_69520_row40_col4, #T_69520_row40_col5, #T_69520_row40_col6, #T_69520_row40_col7, #T_69520_row41_col0, #T_69520_row41_col1, #T_69520_row41_col2, #T_69520_row41_col3, #T_69520_row41_col4, #T_69520_row41_col5, #T_69520_row41_col6, #T_69520_row41_col7, #T_69520_row42_col0, #T_69520_row42_col1, #T_69520_row42_col2, #T_69520_row42_col3, #T_69520_row42_col4, #T_69520_row42_col5, #T_69520_row42_col6, #T_69520_row42_col7, #T_69520_row43_col0, #T_69520_row43_col1, #T_69520_row43_col2, #T_69520_row43_col3, #T_69520_row43_col4, #T_69520_row43_col5, #T_69520_row43_col6, #T_69520_row43_col7, #T_69520_row44_col0, #T_69520_row44_col1, #T_69520_row44_col2, #T_69520_row44_col3, #T_69520_row44_col4, #T_69520_row44_col5, #T_69520_row44_col6, #T_69520_row44_col7, #T_69520_row45_col0, #T_69520_row45_col1, #T_69520_row45_col2, #T_69520_row45_col3, #T_69520_row45_col4, #T_69520_row45_col5, #T_69520_row45_col6, #T_69520_row45_col7 {
  background-color: grey;
}
#T_69520_row22_col0, #T_69520_row22_col1, #T_69520_row22_col2, #T_69520_row22_col3, #T_69520_row22_col4, #T_69520_row22_col5, #T_69520_row22_col6, #T_69520_row22_col7, #T_69520_row23_col0, #T_69520_row23_col1, #T_69520_row23_col2, #T_69520_row23_col3, #T_69520_row23_col4, #T_69520_row23_col5, #T_69520_row23_col6, #T_69520_row23_col7 {
  background-color: yellow;
}
#T_69520_row24_col0, #T_69520_row24_col1, #T_69520_row24_col2, #T_69520_row24_col3, #T_69520_row24_col4, #T_69520_row24_col5, #T_69520_row24_col6, #T_69520_row24_col7 {
  background-color: orange;
}
#T_69520_row46_col0, #T_69520_row46_col1, #T_69520_row46_col2, #T_69520_row46_col3, #T_69520_row46_col4, #T_69520_row46_col5, #T_69520_row46_col6, #T_69520_row46_col7 {
  background-color: red;
}
</style>
<table id="T_69520">
  <thead>
    <tr>
      <th class="blank level0" >&nbsp;</th>
      <th id="T_69520_level0_col0" class="col_heading level0 col0" >timestamp</th>
      <th id="T_69520_level0_col1" class="col_heading level0 col1" >log_level</th>
      <th id="T_69520_level0_col2" class="col_heading level0 col2" >process_id</th>
      <th id="T_69520_level0_col3" class="col_heading level0 col3" >thread_id</th>
...
    </tr>
  </tbody>
</table>

# Lifecycle of an Actor

An actor is a stateful object that can be used to encapsulate state and methods that operate on that state.


## 10,000 feet view

Let's take an example of a simple counter actor. We create an actor handle by calling `Counter.remote()`. 

In [12]:
import ray


@ray.remote
class MyCounter:
    def __init__(self) -> None:
        self.counter = 0

    def increment(self):
        time.sleep(3)
        self.counter += 1

    def get_counter(self):
        return self.counter


my_counter_handle = MyCounter.remote()

We can then call methods on the actor handle to increment the counter and get the current value of the counter. The methods will be executed sequentially against the actor process.

In [13]:
# this will take 3 seconds * 2 = 6 seconds at least
ray.get([my_counter_handle.increment.remote() for _ in range(2)])

(raylet) Spilled 6000 MiB, 6 objects, write throughput 3255 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


[None, None]

In [14]:
ray.get(my_counter_handle.get_counter.remote())

2

Here is a diagram showing the lifecycle of our actor (note that our actor is referred to as a "synchronous" actor)


<img src="actor_simple_.jpeg" height="300">

- A special "create actor" task is executed on the cluster to create the actor process
- The actor process can be thought of as a special worker process
- The actor tasks are executed sequentially on the actor process using a FIFO queue

## 1,000 feet view of ray actors

In this section we will detail the lifecycle of an actor in more detail.

- Actors are always owned by the GCS (global control service), unlike tasks which are owned by the worker process that submitted them
- The GCS maintains an actor table that keeps track of all the actors in the cluster
- Actors hold the resources they need to execute their tasks until they are killed
- Actors can be launched in a detached mode, in which case they do not fate share with a ray driver/job - instead they need to be killed manually

See the below diagram for more details


<img src="actor_centralized.jpeg" height="300">


Our actors can be asynchronous - this is especially useful for actors whose methods are IO bound and whose state can be easily shared and locked if needed

In [15]:
import ray
from asyncio import sleep


@ray.remote
class MyAsyncService:
    def __init__(self) -> None:
        self.fixed_state = 1

    async def run(self):
        await sleep(15)
        return self.fixed_state


my_async_actor_handle = MyAsyncService.remote()

Given the service run is mostly IO bound (sleeping), we can run it asynchronously using an asynchronous actor implementation

In [16]:
%%time

ray.get([my_async_actor_handle.run.remote() for _ in range(2)])

CPU times: user 28 ms, sys: 26.4 ms, total: 54.4 ms
Wall time: 15.3 s


[1, 1]

Here is a diagram visualizing task execution against an asynchroneous actor.

<img src="actor_async.jpeg" height="300">